In [ ]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel
import resend

In [ ]:
load_dotenv(override=True)

In [ ]:
google_api_key = os.getenv('GEMINI_API_KEY')
if google_api_key:
    print(f"Google API Key exists")
else:
    print("Google API Key not set (and this is optional)")


In [ ]:
resend_api_key=os.getenv('RESEND_API_KEY')
if resend_api_key:
    print(f"Resend api key found")
else:
    print("Resend api key not found")

In [ ]:
concise_instructions = "You are a bug summarizer. Write a short, standardized summary of the bug. \
If steps or logs are missing, put 'not provided' .\
Return JSON with fields: title, reproductions_steps[], summary."

developer_instructions = "You are a developer oriented bug formatter.\
if logs/repro/environment are missing, fill with 'not available' or 'unclear'.\
Return JSON with fields: title, reproduction_steps[],stacktrace_analysis,likely_root_cause,debug_steps[],environment(dev|qa|prod) "
stakeholder_instructions = "You write a business friendly view of the bug. \
Return JSON with fields: impact_summary, business_impact "
clarifications_needed = "You need to list down points that needs to be clarified from the user to help solve the reported bug. \
Remember to only list down points that an end user can relate to. No technical points should be list down as the dev team is usually \
aware of the technology being used."
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.No technical structures like json should be there \
It should be well formatted, compelling layout which can be shared with the business user, development team and other stakeholders"

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

In [ ]:
concise_agent=Agent(name="concise_agent",instructions=concise_instructions,model=gemini_model)
developer_agent=Agent(name="developer_agent",instructions=developer_instructions,model=gemini_model)
stakeholder_agent=Agent(name="stakeholder_agent",instructions=stakeholder_instructions,model=gemini_model)
clarification_agent=Agent(name="clarifications_agent",instructions=clarifications_needed,model=gemini_model)
html_agent=Agent(name="HTML_Converter_Agent",instructions=html_instructions,model=gemini_model)

In [ ]:
concise_tool=concise_agent.as_tool(tool_name="concise_tool",tool_description="Generates a concise summary")
developer_tool=developer_agent.as_tool(tool_name="developer_tool",tool_description="Generates developer view")
stakeholder_tool=stakeholder_agent.as_tool(tool_name="stakeholder_tool",tool_description="Generates stakeholder view")
clarification_tool=clarification_agent.as_tool(tool_name="clarification_tool",tool_description="Generates points for clarification")
html_tool=html_agent.as_tool(tool_name="html_converter_tool",tool_description="Convert a text email body to an HTML email body")

In [ ]:
class EvaluationResult(BaseModel):
    primary_draft: str
    reason: str
    needs_human_review: bool

In [ ]:
@function_tool
def evaluator_tool(concise, developer, stakeholder):
    """"Decides which drfat should be the primary one.
    Returns JSON with primary_draft(concise|developer|stakeholder),reason, needs_human_review. """
    if developer.get("reproduction_steps") and "not available" not in developer.get("reproduction_steps",[]):
        choice="developer"
        reason="Developer draft has repro steps"
    elif "impact_summary" in stakeholder:
        choice= "stakeholder"
        reason= "Business-facing impact is clearer"
    else:
        choice="concise"
        reason= "only concise draft usable"
    return {
        "primary_draft":choice,
        "reason":reason,
        "needs_human_review": False
    }

In [ ]:
resend.api_key = resend_api_key
@function_tool
def send_email(to_email,subject,body):
  """
  Sends bug report emai via resend api
  """
  r = resend.Emails.send({
    "from": "onboarding@resend.dev",
    "to": to_email,
    "subject": subject,
    "html": body
  })
  return {"success"}

In [ ]:
manager=Agent(name="manager",model=gemini_model, instructions="""
You are the Bug Report Manager. 
Steps:
1. Call the concise_tool, developer_tool, stakeholder_tool and clarification_tool to generate the drafts
2. Call evaluator_tool with their outputs. 
3. Use the evaluator tool to decide the subject and body. The body should also contain the results of clarification tool in the end if needed. \
    if the evaluator tool fails decide based on the given information decide youself of the subject and body with relevant details 
4. Draft an email as the final bug report.
The report should be from perspective of an issue being reported by the user, therefore do not use I anywhere.The report will have primary draft as the main portion and other drafts should also be there in the mail as important subparts. It should be well \
formatted as email based on the information and no technical structures like json should be used in email. 
5. Use the html_tool to convert the text body into html to ensure formatting and layout of mail content is  maintained. Well formatted points to ensure readability
6. Use the send_email tool to finally send the email""",
tools=[concise_tool,developer_tool,stakeholder_tool,clarification_tool,evaluator_tool,html_tool,send_email])

In [ ]:

# from agents import result

result=await Runner.run(manager,
    input=[
        {"role":"user","content":"Payment Page Crashedw with white screen after submitting upi details. Observed on android 13, one plus 9, version 5.2.0"},
        {"role":"user","content":"Recipient email: ['recipient email id']"}
    ]
)
print(result.final_output)
